### Code for Rstan 

In [ ]:
library(rstan)
library(dplyr)
library(ggplot2)

# Data
setwd("C:/Users/27231/Documents")
tsla <- read.csv("TSLA.csv")
tsla$Date <- as.Date(tsla$Date)
tsla <- tsla %>% arrange(Date)
tsla <- tsla %>% filter(Date >= max(Date) - 365)
tsla <- tsla %>%
  mutate(
    Lag1 = lag(Close),
    Lag2 = lag(Close, 2)
  ) %>%
  na.omit()

n_total <- nrow(tsla)
n_train <- round(n_total * 10 / 12)
train <- tsla[1:n_train, ]
test <- tsla[(n_train + 1):n_total, ]
n_forecast <- nrow(test)

stan_data <- list(
  N = nrow(train),
  K = 2,
  X = as.matrix(train[, c("Lag1", "Lag2")]),
  y = train$Close
)

# Fit Bayesian model
fit <- stan("BSTS.stan", data = stan_data, iter = 1000, chains = 2, seed = 5412)

post <- rstan::extract(fit)
n_sim <- length(post$alpha)

lag1 <- tail(train$Close, 1)
lag2 <- train$Close[nrow(train) - 1]

future_pred <- matrix(NA, nrow = n_sim, ncol = n_forecast)

for (s in 1:n_sim) {
  alpha <- post$alpha[s]
  beta1 <- post$beta1[s]
  beta2 <- post$beta2[s]
  sigma <- post$sigma[s]
  
  l1 <- lag1
  l2 <- lag2
  
  for (t in 1:n_forecast) {
    mu <- alpha + beta1 * l1 + beta2 * l2
    y_sim <- rnorm(1, mu, sigma)
    future_pred[s, t] <- y_sim
    
    l2 <- l1
    l1 <- y_sim
  }
}

# Posterior mean
forecast_mean <- apply(future_pred, 2, mean)

# Forecast plot
forecast_df <- data.frame(
  Date = test$Date,
  Forecast = forecast_mean,
  Actual = test$Close
)

ggplot(forecast_df, aes(x = Date)) +
  geom_line(aes(y = Actual), color = "black") +
  geom_line(aes(y = Forecast), color = "red") +
  labs(title = "Recursive Bayesian Forecast",
       y = "Closing Price", x = "Date") +
  theme_minimal()

# RMSE VS MAE
errors <- forecast_df$Forecast - forecast_df$Actual
rmse <- sqrt(mean(errors^2))
mae <- mean(abs(errors))
rmse
mae

# Trace plot 
traceplot(fit, pars = c("alpha", "beta1", "beta2", "sigma"))


# Calibration check
n_obs <- nrow(train)
y_hat_matrix <- matrix(NA, nrow = n_sim, ncol = n_obs)

for (s in 1:n_sim) {
  mu <- post$alpha[s] +
    train$Lag1 * post$beta1[s] +
    train$Lag2 * post$beta2[s]
  y_hat_matrix[s, ] <- mu
}

y_hat_mean <- apply(y_hat_matrix, 2, mean)

df_calib <- data.frame(
  Fitted = y_hat_mean,
  Actual = train$Close
)

ggplot(df_calib, aes(x = Fitted, y = Actual)) +
  geom_point(color = "red", alpha = 0.6) +
  geom_abline(slope = 1, intercept = 0, color = "black") +
  labs(title = "Calibration Plot (Training Data)",
       x = "Predicted Price", y = "Actual Price") +
  theme_minimal()
